In [ ]:
import os
from configparser import ConfigParser

import anndata
import pandas as pd

import scglue

In [ ]:
PATH = "s06_genomic_tracks"
os.makedirs(PATH, exist_ok=True)

# Read data

In [ ]:
rna = anndata.read_h5ad("s01_preprocessing/rna.h5ad", backed="r")
genes = scglue.genomics.Bed(rna.var.assign(name=rna.var_names))
tss = genes.strand_specific_start_site()

In [ ]:
gene_peak_conn = pd.read_pickle("s04_infer_gene_tf/gene_peak_conn.pkl.gz")

# Plot tracks

In [ ]:
def plot_tracks(gene, tfs):
    vis_peaks = gene_peak_conn.query(f"gene == '{gene}'").loc[:, ["peak", "corr"]]
    vis_peaks["peak_info"] = vis_peaks["peak"].str.split(":")
    vis_peaks["chrom"] = vis_peaks["peak_info"].map(lambda x: x[0])
    vis_peaks["peak_info"] = vis_peaks["peak_info"].map(lambda x: x[1].split("-"))
    vis_peaks["chromStart"] = vis_peaks["peak_info"].map(lambda x: x[0])
    vis_peaks["chromEnd"] = vis_peaks["peak_info"].map(lambda x: x[1])
    vis_peaks = vis_peaks.loc[:, ["chrom", "chromStart", "chromEnd", "peak", "corr"]]
    vis_range = vis_peaks["corr"].abs().max()
    vis_peaks.to_csv(f"{PATH}/vis_peaks.bed", sep="\t", header=False, index=False)

    !grep -w '{gene}$' s01_preprocessing/pchic.annotated_links | cut -f1-7 > {PATH}/pchic.links
    !grep -w '{gene}$' s01_preprocessing/eqtl.annotated_links | cut -f1-7 > {PATH}/eqtl.links
    !grep -w '{gene}$' s04_infer_gene_tf/glue.annotated_links | cut -f1-7 > {PATH}/glue.links
    
    config = ConfigParser()
    config.add_section("spacer")
    link_kws = {
        "links_type": "arcs", "line_width": 1, "line_style": "solid",
        "compact_arcs_level": 2, "use_middle": True, "file_type": "links"
    }
    config["GLUE"] = {
        "file": "s06_genomic_tracks/glue.links",
        "title": "GLUE", "height": 2, "color": "YlGnBu", "min_value": 0, **link_kws
    }
    config["pcHi-C"] = {
        "file": "s06_genomic_tracks/pchic.links",
        "title": "pcHi-C", "color": "darkblue", "height": 1.5, **link_kws
    }
    config["eQTL"] = {
        "file": "s06_genomic_tracks/eqtl.links",
        "title": "eQTL", "color": "darkgreen", "height": 1.5, **link_kws
    }
    bed_kws = {
        "display": "collapsed", "border_color": "none",
        "labels": False, "file_type": "bed"
    }
    config["ATAC"] = {
        "file": "s06_genomic_tracks/vis_peaks.bed",
        "title": "ATAC", "color": "bwr", "height": 1,
        "min_value": -vis_range, "max_value": vis_range, **bed_kws
    }
    for tf in tfs:
        config[f"{tf} ChIP"] = {
            "file": f"../../data/chip/ENCODE/TF-human/targets-GRCh38/{tf}.bed.gz",
            "title": f"{tf} ChIP", "color": "darkred", "height": 0.7, **bed_kws
        }
    config["Genes"] = {
        "file": "../../data/genome/gencode.v35.chr_patch_hapl_scaff.annotation.gtf.gz",
        "title": "Genes", "prefered_name": "gene_name", "merge_transcripts": True,
        "fontsize": 10, "height": 5, "labels": True, "max_labels": 100,
        "all_labels_inside": True, "style": "UCSC", "file_type": "gtf"
    }
    config["x-axis"] = {"fontsize": 12}
    with open(f"{PATH}/tracks-{gene}.ini", "w") as f:
        config.write(f)

    chrom, chromStart, chromEnd = tss.loc[gene, ["chrom", "chromStart", "chromEnd"]]
    chromStart -= 200000
    chromEnd += 200000
    !pyGenomeTracks --tracks {PATH}/tracks-{gene}.ini --region {chrom}:{chromStart}-{chromEnd} \
        -t 'Target gene: {gene}' --dpi 600 --width 22 --fontSize 10 \
        --outFileName {PATH}/tracks-{gene}.pdf 2> /dev/null
    !rm {PATH}/vis_peaks.bed {PATH}/*.links

In [ ]:
GENE_TF_DICT = {
    "FCER2": ["PAX5", "IRF4", "EBF1", "BCL11A", "MEF2C"],
    "NCF2": ["SPI1"],
    "IFIT3": ["SPI1", "HMGA2"],
    "ITGAX": ["SPI1", "IRF4"],
    "CYBB": ["SPI1"],

    "CD83": ["BCL11A", "PAX5", "RELB"],
    "CCL4": ["BATF", "TBX21"],
    "ATXN1": ["BATF", "TBX21"],
    "KLRD1": ["BATF", "TBX21"],
    "IL2RB": ["TBX21"],
    "GBP2": ["IRF1"],
    "GCH1": ["IRF1"]
}

In [ ]:
for gene, tfs in GENE_TF_DICT.items():
    print(f"Dealing with {gene}...")
    plot_tracks(gene, tfs)